In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error 
from keras.layers import LSTM, Dense
from keras.models import Sequential

In [ ]:
Aquifier_Auser = pd.read_csv('../input/acea-water-prediction/Aquifer_Auser.csv')

In [ ]:
print("Data shape :" ,Aquifier_Auser.shape)

In [ ]:
Aquifier_Auser.info()


In [ ]:
sns.heatmap(Aquifier_Auser.isnull())


In [ ]:
Aquifier_Auser.iloc[2850:2900,:]

In [ ]:
Aquifier_Auser_1 = Aquifier_Auser.copy()
Aquifier_Auser_1 = Aquifier_Auser.iloc[2859:,:]

In [ ]:
sns.heatmap(Aquifier_Auser_1.isnull())


In [ ]:
Aquifier_Auser_1.reset_index()

In [ ]:
Aquifier_Auser_1.interpolate(method ='pchip', limit_direction ='forward', inplace=True)

In [ ]:
Aquifier_Auser_1.fillna(Aquifier_Auser_1.mean())
Aquifier_Auser_1.drop(Aquifier_Auser_1.tail(2).index,inplace=True)

In [ ]:
Aquifier_Auser_1.tail()

In [ ]:
sns.heatmap(Aquifier_Auser_1.isnull())



In [ ]:

Aquifier_Auser_1.loc[Aquifier_Auser_1['Depth_to_Groundwater_CoS'] == 0,'Depth_to_Groundwater_CoS'] = np.nan
Aquifier_Auser_1.loc[Aquifier_Auser_1['Depth_to_Groundwater_SAL'] == 0,'Depth_to_Groundwater_SAL'] = np.nan
Aquifier_Auser_1.loc[Aquifier_Auser_1['Depth_to_Groundwater_LT2'] == 0,'Depth_to_Groundwater_LT2'] = np.nan

In [ ]:
Aquifier_Auser_1['Depth_to_Groundwater_CoS'].interpolate(method ='pchip', limit_direction ='forward', inplace=True)

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Auser_1["Rainfall_Piaggione"])
plt.title("Rainfall_Piaggione")

In [ ]:
Aquifier_Auser_1['Day'] =Aquifier_Auser_1['Date'].str.split('/').str[0]
Aquifier_Auser_1['Month'] = Aquifier_Auser_1['Date'].str.split('/').str[1]
Aquifier_Auser_1['Year'] = Aquifier_Auser_1['Date'].str.split('/').str[2]

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Auser_1['Month'],Aquifier_Auser_1['Depth_to_Groundwater_SAL'])
plt.xlabel('Month')
plt.ylabel('Depth_to_Groundwater_SAL')

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Auser_1['Month'],Aquifier_Auser_1['Depth_to_Groundwater_CoS'])
plt.xlabel('Month')
plt.ylabel('Depth_to_Groundwater_CoS')

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Auser_1['Month'],Aquifier_Auser_1['Depth_to_Groundwater_LT2'])
plt.xlabel('Month')
plt.ylabel('Depth_to_Groundwater_LT2')

In [ ]:
sns.heatmap(Aquifier_Auser_1.corr());


In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Auser_1["Depth_to_Groundwater_LT2"])
plt.title("Depth_to_Groundwater_LT2")


In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Auser_1["Depth_to_Groundwater_CoS"])
plt.title("Depth_to_Groundwater_CoS")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Auser_1["Depth_to_Groundwater_SAL"])
plt.title("Depth_to_Groundwater_SAL")

In [ ]:
Aquifier_Auser_1.info()


In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardizing the features
Aquifier_Auser_1_T = Aquifier_Auser_1.T
x = StandardScaler().fit_transform(Aquifier_Auser_1_T.iloc[1:11,:])

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents)

In [ ]:
principalDf.head()


In [ ]:
eigvals = pca.explained_variance_ratio_
eigvals/np.sum(eigvals) * 100

In [ ]:
plt.figure(figsize = (8,8))
plt.bar(list(range(10)),eigvals/np.sum(eigvals) * 100)
plt.show()

In [ ]:
Aquifier_Auser_1.columns

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

features = [ 'Rainfall_Gallicano', 'Rainfall_Pontetetto',
       'Rainfall_Monte_Serra', 'Rainfall_Orentano', 'Rainfall_Borgo_a_Mozzano',
       'Rainfall_Piaggione', 'Rainfall_Calavorno', 'Rainfall_Croce_Arcana',
       'Rainfall_Tereglio_Coreglia_Antelminelli',
       'Rainfall_Fabbriche_di_Vallico']
features1 = list(range(21))

ax.scatter(principalDf.loc[:, 0]
            , principalDf.loc[:, 1]
            , s = 50)

for i, txt in enumerate(features):
    ax.annotate(txt, (principalDf.loc[i, 0], principalDf.loc[i, 1]))

ax.grid()

In [ ]:
x1 = StandardScaler().fit_transform(Aquifier_Auser_1_T.iloc[16:20,:])
pca1 = PCA()
principalComponents1 = pca1.fit_transform(x1)
principalDf1 = pd.DataFrame(data = principalComponents1)

In [ ]:
eigvals1 = pca1.explained_variance_ratio_
eigvals1/np.sum(eigvals1) * 100

In [ ]:
plt.figure(figsize = (8,8))
plt.bar(list(range(4)),eigvals1/np.sum(eigvals1) * 100)
plt.show()

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)

features2 = [  'Temperature_Orentano', 'Temperature_Monte_Serra',
       'Temperature_Ponte_a_Moriano', 'Temperature_Lucca_Orto_Botanico']


ax.scatter(principalDf1.loc[:, 0]
            , principalDf1.loc[:, 1]
            , s = 50)

for i, txt in enumerate(features2):
    ax.annotate(txt, (principalDf1.loc[i, 0], principalDf1.loc[i, 1]))

ax.grid()

# Aquifier_Petrignano

**Loading Data**

In [ ]:
Aquifier_Petrignano = pd.read_csv('../input/acea-water-prediction/Aquifer_Petrignano.csv')

**Exploratory analysis**

In [ ]:
print("Data shape :" ,Aquifier_Petrignano.shape)
Aquifier_Petrignano.info()


In [ ]:
Aquifier_Petrignano.head(100)


In [ ]:
sns.heatmap(Aquifier_Petrignano.isnull())

**Getting rid of Nan values**

In [ ]:
Aquifier_Petrignano_1 = Aquifier_Petrignano.copy()
Aquifier_Petrignano_1 = Aquifier_Petrignano.iloc[1024:,:]

In [ ]:
sns.heatmap(Aquifier_Petrignano_1.isnull())

**Interpolating missing values**

In [ ]:
Aquifier_Petrignano_1.interpolate(method ='pchip', limit_direction ='forward', inplace=True)

In [ ]:
sns.heatmap(Aquifier_Petrignano_1.isnull())

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Petrignano_1["Depth_to_Groundwater_P24"])
plt.title("Depth_to_Groundwater_P24")


In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Petrignano_1["Depth_to_Groundwater_P25"])
plt.title("Depth_to_Groundwater_P25")


In [ ]:
Aquifier_Petrignano_1['Day'] =Aquifier_Petrignano_1['Date'].str.split('/').str[0]
Aquifier_Petrignano_1['Month'] = Aquifier_Petrignano_1['Date'].str.split('/').str[1]
Aquifier_Petrignano_1['Year'] = Aquifier_Petrignano_1['Date'].str.split('/').str[2]

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Petrignano_1['Month'],Aquifier_Petrignano_1['Depth_to_Groundwater_P24'])
plt.xlabel('Month')
plt.ylabel('Depth_to_Groundwater_P24')

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Petrignano_1['Month'],Aquifier_Petrignano_1['Depth_to_Groundwater_P25'])
plt.xlabel('Month')
plt.ylabel('Depth_to_Groundwater_P25')

# Aquifier_Doganella


In [ ]:
Aquifier_Doganella = pd.read_csv('../input/acea-water-prediction/Aquifer_Doganella.csv')



In [ ]:
print("Data shape :" ,Aquifier_Doganella.shape)
Aquifier_Doganella.info()


In [ ]:
sns.heatmap(Aquifier_Doganella.isnull())


In [ ]:
Aquifier_Doganella.iloc[4650:4680,:]

In [ ]:

Aquifier_Doganella_1 = Aquifier_Doganella.copy()
Aquifier_Doganella_1 = Aquifier_Doganella_1.iloc[4663:,:]

In [ ]:
sns.heatmap(Aquifier_Doganella_1.isnull())


In [ ]:
Aquifier_Doganella_1.interpolate(method ='pchip', limit_direction ='forward', inplace=True)

In [ ]:
sns.heatmap(Aquifier_Doganella_1.isnull())


In [ ]:
Aquifier_Doganella_1.dropna(inplace= True)

In [ ]:
sns.heatmap(Aquifier_Doganella_1.isnull())


In [ ]:
Aquifier_Doganella_1['Day'] =Aquifier_Doganella_1['Date'].str.split('/').str[0]
Aquifier_Doganella_1['Month'] = Aquifier_Doganella_1['Date'].str.split('/').str[1]
Aquifier_Doganella_1['Year'] = Aquifier_Doganella_1['Date'].str.split('/').str[2]

In [ ]:
plt.figure(figsize = (12,6))
plt.bar(Aquifier_Doganella_1['Month'],Aquifier_Doganella_1['Volume_Pozzo_1'])
plt.xlabel('Month')
plt.ylabel('Volume_Pozzo_1')

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Volume_Pozzo_1"])
plt.title("Volume_Pozzo_1")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Temperature_Monteporzio"])
plt.title("Temperature_Monteporzio ")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_1"])
plt.title("Depth_to_Groundwater_Pozzo_1")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_2"])
plt.title("Depth_to_Groundwater_Pozzo_2")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_3"])
plt.title("Depth_to_Groundwater_Pozzo_3")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_4"])
plt.title("Depth_to_Groundwater_Pozzo_4")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_5"])
plt.title("Depth_to_Groundwater_Pozzo_5")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_6"])
plt.title("Depth_to_Groundwater_Pozzo_6")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_7"])
plt.title("Depth_to_Groundwater_Pozzo_7")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_8"])
plt.title("Depth_to_Groundwater_Pozzo_8")

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Doganella_1["Depth_to_Groundwater_Pozzo_9"])
plt.title("Depth_to_Groundwater_Pozzo_9")

In [ ]:
sns.heatmap(Aquifier_Doganella_1.corr());


In [ ]:
Aquifier_Doganella_1.head(10)

# Aquifier_Luco 


In [ ]:
Aquifier_Luco = pd.read_csv('../input/acea-water-prediction/Aquifer_Luco.csv')


In [ ]:
print("Data shape :" ,Aquifier_Luco.shape)
Aquifier_Luco.info()


In [ ]:
sns.heatmap(Aquifier_Luco.isnull())

In [ ]:
Aquifier_Luco_1 = Aquifier_Luco.copy()
Aquifier_Luco_1 = Aquifier_Luco_1.drop(['Rainfall_Siena_Poggio_al_Vento','Depth_to_Groundwater_Pozzo_3','Depth_to_Groundwater_Pozzo_4','Depth_to_Groundwater_Pozzo_1','Rainfall_Ponte_Orgia','Rainfall_Mensano','Rainfall_Monticiano_la_Pineta','Volume_Pozzo_1','Volume_Pozzo_4','Volume_Pozzo_3'],axis = 1)

In [ ]:
Aquifier_Luco_1 = Aquifier_Luco_1.copy()
Aquifier_Luco_1 = Aquifier_Luco_1.iloc[2973:,:]

In [ ]:
sns.heatmap(Aquifier_Luco_1.isnull())

In [ ]:
Aquifier_Luco_1.interpolate(method ='pchip', limit_direction ='forward', inplace=True)

In [ ]:
Aquifier_Luco_1.dropna(inplace = True)

In [ ]:
Aquifier_Luco_1.tail(500)

In [ ]:
Aquifier_Luco_1 = Aquifier_Luco_1.reset_index()
Aquifier_Luco_1 = Aquifier_Luco_1.iloc[:2605,:]

In [ ]:
sns.heatmap(Aquifier_Luco_1.isnull())

In [ ]:
plt.figure(1,figsize = (10,5))
plt.plot(Aquifier_Luco_1["Depth_to_Groundwater_Podere_Casetta"])
plt.title("Depth_to_Groundwater_Podere_Casetta")